In [ ]:
# Loading the necessary libraries
library(ggplot2)
library(tidyverse)
library(gridExtra)
library(plotly)
library(corrplot)
library(reshape2)
library(FactoMineR) 
library(factoextra)
library(glmnet) 
library(ggfortify)
library(pROC)
library(ROCR)


person_age 	Age of the person 	Float
person_gender 	Gender of the person 	Categorical
person_education 	Highest education level 	Categorical
person_income 	Annual income 	Float
person_emp_exp 	Years of employment experience 	Integer
person_home_ownership 	Home ownership status (e.g., rent, own, mortgage) 	Categorical
loan_amnt 	Loan amount requested 	Float
loan_intent 	Purpose of the loan 	Categorical
loan_int_rate 	Loan interest rate 	Float
loan_percent_income 	Loan amount as a percentage of annual income 	Float
cb_person_cred_hist_length 	Length of credit history in years 	Float
credit_score 	Credit score of the person 	Integer
previous_loan_defaults_on_file 	Indicator of previous loan defaults 	Categorical
loan_status (target variable) 	Loan approval status: 1 = approved; 0 = rejected 	Integer

In [ ]:
data <- read.csv("loan_data.csv")
head(data)
summary(data)

In [ ]:
data[, "person_gender"] <- as.factor(data[, "person_gender"])
data[, "person_education"] <- as.factor(data[, "person_education"])
data[, "person_home_ownership"] <- as.factor(data[, "person_home_ownership"])
data[, "loan_intent	"] <- as.factor(data[, "loan_intent"])
data[, "previous_loan_defaults_on_file"] <- as.factor(data[, "previous_loan_defaults_on_file"])
data[, "loan_status"] <- as.factor(data[, "loan_status"])

In [ ]:
summary(data)

In [ ]:
sum(is.na(data))  # Đếm tổng số giá trị NA trong toàn bộ dataframe


In [ ]:
g3<-ggplot(data,aes(x=person_age))+geom_histogram(aes(y=..density..))+geom_density(alpha=.2, col="blue") 
g4<-ggplot(data,aes(x=loan_percent_income))+geom_histogram(aes(y=..density..))+geom_density(alpha=.2, col="blue") 
g5<-ggplot(data,aes(x=loan_amnt))+geom_histogram(aes(y=..density..))+geom_density(alpha=.2, col="blue") 
g6<-ggplot(data,aes(x=loan_int_rate))+geom_histogram(aes(y=..density..))+geom_density(alpha=.2, col="blue") 
g7<-ggplot(data,aes(x=cb_person_cred_hist_length))+geom_histogram(aes(y=..density..))+geom_density(alpha=.2, col="blue") 
g8<-ggplot(data,aes(x=credit_score))+geom_histogram(aes(y=..density..))+geom_density(alpha=.2, col="blue") 

grid.arrange(g3,g4,g5,g6,g7,g8,ncol=3)
rm(g1,g2,g3,g4,g5,g6,g7,g8)

In [ ]:
data[, "person_age"] <- log(data[, "person_age"])

In [ ]:
g3<-ggplot(data,aes(x=person_age))+geom_histogram(aes(y=..density..))+geom_density(alpha=.2, col="blue") 
g4<-ggplot(data,aes(x=loan_percent_income))+geom_histogram(aes(y=..density..))+geom_density(alpha=.2, col="blue") 
g5<-ggplot(data,aes(x=loan_amnt))+geom_histogram(aes(y=..density..))+geom_density(alpha=.2, col="blue") 
g6<-ggplot(data,aes(x=loan_int_rate))+geom_histogram(aes(y=..density..))+geom_density(alpha=.2, col="blue") 
g7<-ggplot(data,aes(x=cb_person_cred_hist_length))+geom_histogram(aes(y=..density..))+geom_density(alpha=.2, col="blue") 
g8<-ggplot(data,aes(x=credit_score))+geom_histogram(aes(y=..density..))+geom_density(alpha=.2, col="blue") 

grid.arrange(g3,g4,g5,g6,g7,g8,ncol=3)
rm(g3,g4,g5,g6,g7,g8)

In [ ]:
library(corrplot)
corrplot(cor(data[,c(1,4,5,7,9,10,11,12)]),method="ellipse")
boxplot(data[4])

In [ ]:
library(FactoMineR)
acp <- PCA(data[, c(1,4,5,7,9,10,11,12)], scale.unit = TRUE,
           graph = FALSE, ncp = 8)
# Eigenvalue decay
library(factoextra)
g1<-fviz_eig(acp, addlabels = TRUE, ylim = c(0, 40))
library(reshape2)
g2<-ggplot(melt(acp$ind$coord),aes(x=Var2,y=value))+
  geom_boxplot()+
  xlab("")
grid.arrange(g1,g2,ncol=2)
# 
library(corrplot)
corrplot(acp$var$cor, is.corr=FALSE,method="ellipse")

In [ ]:
barplot(data[person_age])